In [247]:
import pandas as pd

# 가스 데이터
gas = pd.read_csv('./data/한국가스공사_월간 시도별 도시가스 판매현황_20201231.csv', encoding='cp949')

# gas.info()
# gas.head()
# gas.tail()

# 주어진 가스 데이터는 1989년 1월 ~ 2020년 12월 도시가스판매현황

In [248]:
# 날씨 데이터
weather = pd.read_csv('./data/2000-01~2021-11m.csv', encoding='cp949')

# weather.info()
# weather.head()
# weather.tail()

# 주어진 가스 데이터는 2000년 1월 ~ 2021년 11월 도시가스판매현황

In [249]:
# 날씨, 가스 데이터를 inner join 할때 year, month를 중심으로 했는데 숫자형 데이터로 변환하지 않고 object로 놔뒀더니 데이터가 일치하지 않으면서 데이터가 하나도 저장되지 않는 현상이 발생했기 때문에 연-월 데이터를 분리하면서 숫자형 데이터로 변환해 데이터가 일치되도록 만들었다
weather['year'] = pd.to_numeric(weather['일시'].str.split('-').str.get(0))
weather['month'] = pd.to_numeric(weather['일시'].str.split('-').str.get(1))

weather['meanTemp'] = weather['평균기온(°C)']
weather['meanHighTemp'] = weather['평균최고기온(°C)']
weather['meanLowTemp'] = weather['평균최저기온(°C)']
# weather['highTemp'] = weather['최고기온(°C)']
# weather['lowTemp'] = weather['최저기온(°C)']

gas['year'] = pd.to_numeric(gas['연'])
gas['month'] = pd.to_numeric(gas['월'])
gas['usageGas'] = gas['서울']

In [250]:
# 데이터 표준화하기위해 참고한 식

# 정규분포를 이용한 데이터표준화
# normalization_df = (df - df.mean())/df.std()
# normalization_df.head()

# Min-Max 데이터표준화
# normalization_df = (df - df.mean())/df.std()
# normalization_df.head()

In [251]:
# 정규분포를 이용한 데이터표준화
# 가스사용량이 온도 데이터에 비해 과도하게 크기때문에 표준화 없이 학습을 시키면 loss가 너무 크게 나타났다

weather['meanTemp'] = (weather['meanTemp'] - weather['meanTemp'].mean())/weather['meanTemp'].std()
weather['meanHighTemp'] = (weather['meanHighTemp'] - weather['meanHighTemp'].mean())/weather['meanHighTemp'].std()
weather['meanLowTemp'] = (weather['meanLowTemp'] - weather['meanLowTemp'].mean())/weather['meanLowTemp'].std()

# 표준정규분포값으로 예측된 가스사용량 데이터를 다시 사용량으로 변환하기 위해 변수에 저장
usageMean = gas['usageGas'].mean()
usageStd = gas['usageGas'].std()

gas['usageGas'] = (gas['usageGas'] - gas['usageGas'].mean())/gas['usageGas'].std()

In [252]:
# 가스 데이터에서 year, month, usageGas 만 따로 분리
gas = gas.loc[:,['year','month','usageGas']]

In [253]:
# year, month를 인덱스로 설정
gas.set_index(['year','month'], inplace=True)

In [254]:
gas.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 384 entries, (1989, 1) to (2020, 12)
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   usageGas  384 non-null    float64
dtypes: float64(1)
memory usage: 5.5 KB


In [255]:
# 날씨 데이터에서 year, month, meanTemp, meanHighTemp, meanLowTemp 만 따로 분리
weather = weather.loc[:,['year','month','meanTemp','meanHighTemp','meanLowTemp']]

In [256]:
# year, month를 인덱스로 설정
weather.set_index(['year','month'], inplace=True)

In [257]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 263 entries, (2000, 1) to (2021, 11)
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   meanTemp      263 non-null    float64
 1   meanHighTemp  263 non-null    float64
 2   meanLowTemp   263 non-null    float64
dtypes: float64(3)
memory usage: 7.9 KB


In [258]:
gas.head()

usageGas
year month          
1989 1     -1.172237
     2     -1.189642
     3     -1.204206
     4     -1.254563
     5     -1.267105

In [259]:
weather.head()

meanTemp  meanHighTemp  meanLowTemp
year month                                     
2000 1     -1.511800     -1.577532    -1.468768
     2     -1.471669     -1.487794    -1.478703
     3     -0.669059     -0.610351    -0.733553
     4     -0.107231     -0.061949    -0.177174
     5      0.454597      0.516366     0.379205

In [260]:
# 날씨와 가스 데이터를 inner Join
# axis=1 설정은 열단위 join 하는 설정
dataSet = pd.concat([weather,gas], join='inner',axis=1)

# dataSet.info()
# dataSet.head()
# dataSet.tail()

In [261]:
# 독립변수는 평균온도, 평균최고온도, 평균최저온도
# 종속변수는 가스사용량

# 훈련데이터는 2000 ~ 2019년 데이터
trainX = dataSet.iloc[:240,:].loc[:,['meanTemp','meanHighTemp','meanLowTemp']]
trainY = dataSet.iloc[:240,:].loc[:,'usageGas']

# 시험데이터는 2020년 데이터
testX = dataSet.iloc[240:,:].loc[:,['meanTemp','meanHighTemp','meanLowTemp']]
testY = dataSet.iloc[240:,:].loc[:,'usageGas']

In [262]:
import tensorflow as tf

# 모델 객체 생성
model = tf.keras.Sequential()

# 레이어 1층
model.add(tf.keras.layers.Dense(units=3, input_dim=3))
model.add(tf.keras.layers.Activation('sigmoid'))

# 레이어 2층
model.add(tf.keras.layers.Dense(units=1))
model.add(tf.keras.layers.Activation('linear'))

# activation 함수 - loss 함수 조합 
# linear - mse
# sigmoid - binary_crossentropy
# softmax - categorical_crossentropy

In [263]:
# 마지막 레이어가 linear였기 때문에 loss 함수를 mse로 설정
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=1e-3))
model.summary()

history = model.fit(trainX, trainY, epochs=1000) # 학습

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 3)                 12        
_________________________________________________________________
activation_20 (Activation)   (None, 3)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 4         
_________________________________________________________________
activation_21 (Activation)   (None, 1)                 0         
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
8/8 [==============================] - 0s 464us/step - loss: 2.0676
Epoch 2/1000
8/8 [==============================] - 0s 375us/step - loss: 2.0057
Epoch 3/1000
8/8 [==============================] - 0s 375us/step - loss: 1.9446
Epoch 4/1000

In [264]:
# 시험 데이터로 예측값을 도출
y_pred = model.predict(testX)

In [265]:
from sklearn.metrics import r2_score
r2_score(testY, y_pred)
# 시험데이터 종속변수(이미 존재하는 데이터)와 기계학습모델에 독립변수를 넣어 나온 예측값과 비교하여 R2값을 도출
# 95% 이상 정확성 나온다

0.9538734987691297

In [266]:
# 다른 기계학습에서 예측한 날씨 데이터 가져오기
predict_weather = pd.read_csv('./data/predict_weather.csv', encoding='utf8')

predict_weather.info()
# predict_weather.head()
# predict_weather.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   일시          12 non-null     object 
 1   평균기온(°C)    12 non-null     float64
 2   평균최고기온(°C)  12 non-null     float64
 3   평균최저기온(°C)  12 non-null     float64
dtypes: float64(3), object(1)
memory usage: 512.0+ bytes


In [267]:
# 가져온 예측 날씨 데이터 전처리
predict_weather['year'] = pd.to_numeric(predict_weather['일시'].str.split('-').str.get(0))
predict_weather['month'] = pd.to_numeric(predict_weather['일시'].str.split('-').str.get(1))
predict_weather['meanTemp'] = predict_weather['평균기온(°C)']
predict_weather['meanHighTemp'] = predict_weather['평균최고기온(°C)']
predict_weather['meanLowTemp'] = predict_weather['평균최저기온(°C)']

In [268]:
# 날씨 데이터 표준화 - 날씨에서 가스데이터 도출하는 모델이 표준화되어 작동했기 때문
predict_weather['meanTemp'] = (predict_weather['meanTemp'] - predict_weather['meanTemp'].mean())/predict_weather['meanTemp'].std()
predict_weather['meanHighTemp'] = (predict_weather['meanHighTemp'] - predict_weather['meanHighTemp'].mean())/predict_weather['meanHighTemp'].std()
predict_weather['meanLowTemp'] = (predict_weather['meanLowTemp'] - predict_weather['meanLowTemp'].mean())/predict_weather['meanLowTemp'].std()

In [269]:
predictX = predict_weather.loc[:,['meanTemp','meanHighTemp','meanLowTemp']]
predictX

,meanTemp,meanHighTemp,meanLowTemp
0,1.348244,1.347950,1.348539
1,1.230561,1.230274,1.230848
2,0.783178,0.782858,0.783498
3,0.125972,0.125673,0.126271
4,-0.564967,-0.565165,-0.564769
5,-1.104512,-1.104585,-1.104440
6,-1.348105,-1.348098,-1.348113
7,-1.230485,-1.230449,-1.230521
8,-0.783173,-0.783089,-0.783257
9,-0.126026,-0.125803,-0.126249


In [270]:
predictY = model.predict(predictX) # 새롭게 예측된 가스 사용량 (표준정규분포값)

In [271]:
predictY = predictY*usageStd + usageMean # 표준정규분포 되돌리기
print(predictY)

[[147476.1 ]
 [155182.02]
 [194329.34]
 [296140.34]
 [479710.8 ]
 [644990.94]
 [709727.4 ]
 [679686.56]
 [547915.75]
 [354342.2 ]
 [221016.6 ]
 [164462.22]]
